# Name : Shusmoy Chowdhury
# Partner Name: Marium Binta Ibrahim Ema
# HW : Building a Decision Tree with ID3 and testing the tree
# Run Command: First Click in Kernel And Choose Restart and Run all. In the last block you can update the test_case variable to test the decision tree.
# Loading the training Data

In [9]:
'''
 Name : Shusmoy Chowdhury
 Partner Name: Marium Binte Ibrahim Ema
 HW : Building a Decision Tree with ID3 and testing the tree
 Run Command: First Click in Kernel And Choose Restart and Run all. In the last block you can update the test_case variable to test the decision tree.
'''

'\n Name : Shusmoy Chowdhury\n Partner Name: Marium Binte Ibrahim Ema\n HW : Building a Decision Tree with ID3 and testing the tree\n Run Command: First Click in Kernel And Choose Restart and Run all. In the last block you can update the test_case variable to test the decision tree.\n'

In [10]:
import pandas as pd
import numpy as np
training_data = [
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False)
]
data=[]
class_level=[]
for i in range(len(training_data)):
    data.append(training_data[i][0])
    class_level.append(training_data[i][1])
    
df=pd.DataFrame(data)
df['class']=class_level
df.head()

,level,lang,tweets,phd,class
0,Senior,Java,no,no,False
1,Senior,Java,no,yes,False
2,Mid,Python,no,no,True
3,Junior,Python,no,no,True
4,Junior,R,yes,no,True


# Finding total entropy of a tree

In [11]:
def find_total_entropy(data):
    values=data['class'].unique()
    e=0
    for v in values:
        prob = data['class'].value_counts()[v]/len(data['class'])
        e += -prob*np.log2(prob)
    return e
find_total_entropy(df)   

0.9402859586706311

# Finding entropy of an attribute of the tree

In [12]:
def find_attribute_entropy(data,attribute):
    class_values=data['class'].unique()
    attribute_values=data[attribute].unique()
    te=0
    for values in attribute_values:
        e=0
        for c in class_values:
            num = len(data[attribute][data[attribute]==values][data['class'] ==c])
            den = len(data[attribute][data[attribute]==values])
            prob = num/(den)
            if(num==0):
                e=0
                break
            e += -prob*np.log2(prob)
        prob2 = den/len(data)
        te += -prob2*e
    return abs(te)

find_attribute_entropy(df,'level')
        

0.6935361388961918

# Finding information gain and choosing the node to be selected based on the information gain

In [13]:
def tree_node(data):
    entropy=[]
    gain=[]
    for k in data.keys()[:-1]:
        gain.append(find_total_entropy(data)-find_attribute_entropy(data,k))
    return data.keys()[:-1][np.argmax(gain)] 

tree_node(df)

'level'

# Find the subtree/subtable for an attribute value

In [14]:
def find_subtree(data,attribute,value):
    return data[data[attribute] == value].reset_index(drop=True)
find_subtree(df,'level','Mid')

,level,lang,tweets,phd,class
0,Mid,Python,no,no,True
1,Mid,R,yes,yes,True
2,Mid,Python,no,yes,True
3,Mid,Java,yes,no,True


# Building the Decision Tree

In [15]:
def build_tree(data):
    node = tree_node(data)
    attr = np.unique(data[node])              
    tree={}
    tree[node] = {}
    for value in attr:
        subtree = find_subtree(data,node,value)
        label,counts = np.unique(subtree['class'],return_counts=True)                        
        if len(counts)==1:
            tree[node][value] = label[0]                                                    
        else:
            tree[node][value] = build_tree(subtree) 
    tree[node][None] = df[[node,'class']]['class'].value_counts().idxmax()
    return tree
tree=build_tree(df)
print(tree)

{'level': {'Junior': {'phd': {'no': True, 'yes': False, None: True}}, 'Mid': True, 'Senior': {'tweets': {'no': False, 'yes': True, None: True}}, None: True}}


# Testing the tree

In [16]:
test_case={"level": "Intern","tweets":"yes"} 
def test_tree(test, tree):
    attribute = next(iter(tree)) 
    if attribute not in test.keys():
         return tree[attribute][None]
    if test[attribute] in tree[attribute].keys():
        result = tree[attribute][test[attribute]]
        if isinstance(result, dict):
            return test_tree(test, result)
        else:
            return result
    else:
        return tree[attribute][None]
ans = test_tree(test_case, tree)
print(ans)


True
